<a href="https://colab.research.google.com/github/EvidenceN/DS-Unit-2-Linear-Models/blob/master/%20module3-ridge-regression/Evidence.N.%20Answers_Assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [165]:
# sale price was more than $100 thousand and less than $2 million.
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


Objective is to predict the price of one family homes. One family homes is located under building class category. 

In [166]:
# create into subset where sale price was more than $100 thousand and 
# less than $2 million.
df = df[(df['SALE_PRICE']>100000) & (df['SALE_PRICE']< 2000000)]
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
66,1,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,1347,18,NaN,D4,"345 EAST 54TH ST, 3B",NaN,10022.0,0.0,0.0,0.0,0,0.0,1960.0,2,D4,330000,01/02/2019
67,1,UPPER EAST SIDE (79-96),10 COOPS - ELEVATOR APARTMENTS,2,1491,62,NaN,D4,"16 EAST 80TH STREET, 2A",NaN,10075.0,0.0,0.0,0.0,0,0.0,1925.0,2,D4,600000,01/02/2019
71,1,UPPER WEST SIDE (59-79),13 CONDOS - ELEVATOR APARTMENTS,2,1171,2200,NaN,R4,"240 RIVERSIDE BOULEVARD, 4 F",4 F,10069.0,1.0,0.0,1.0,0,827.0,2004.0,2,R4,1250000,01/02/2019


In [0]:
# Do train/test split. Use data from January — March 2019 to train. 
# Use data from April 2019 to test.

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)

# Training data from january to march; Testing data from April. 
train = df[df['SALE_DATE'].dt.month < 4]
test = df[df['SALE_DATE'].dt.month == 4]

In [168]:
import pandas_profiling
pandas_profiling.ProfileReport(train)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/plot.py:45: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)
/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)
/usr/local/lib/python3.6/dist-packages/pandas_profiling/plot.py:131: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig_cor, a

Number of variables,22
Number of observations,10442
Total Missing (%),8.2%
Total size in memory,1.8 MiB
Average record size in memory,176.0 B
Numeric,11
Categorical,9
Boolean,0
Date,1
Text (Unique),0
Rejected,1


In [169]:
print(train.shape)
train.dtypes

(10442, 21)


BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                float64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
APARTMENT_NUMBER                          object
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                          object
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE           

In [170]:
#  Objective: Do one-hot encoding of categorical features.

# finding high cardinality categories. 

train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
BOROUGH,10442,5,4,3608,NaT,NaT
TAX_CLASS_AT_PRESENT,10442,10,1,4949,NaT,NaT
NEIGHBORHOOD,10442,11,OTHER,8811,NaT,NaT
BUILDING_CLASS_CATEGORY,10442,40,01 ONE FAMILY DWELLINGS,2507,NaT,NaT
SALE_DATE,10442,80,2019-01-31 00:00:00,285,2019-01-01,2019-03-30
BUILDING_CLASS_AT_PRESENT,10442,96,D4,2110,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,10442,96,D4,2110,NaT,NaT
APARTMENT_NUMBER,2136,826,2A,42,NaT,NaT
LAND_SQUARE_FEET,10417,1993,0,4061,NaT,NaT
ADDRESS,10442,10397,N/A BAY STREET,4,NaT,NaT


In [0]:
# defining the target
target = 'SALE_PRICE'

# Exclude high cardinality features. Limit chosen because of the feature we are
# interested in which is building_class_category. 

max_cardinality = 41 # High cardinality limit

# specify high cardinality columns.
high_cardinality = [col for col in train.select_dtypes(exclude='number') if 
                    df[col].nunique() > max_cardinality]
# or high_cardinality = ['list of column names', 'more names']

# dropping high cardinality. # drop ease-ment because it is mostly NAN's
features = train.columns.drop([target] + high_cardinality + ['EASE-MENT'])

# assigning training and testing x and y features
x_train = train[features]
y_train = train[target]
x_test =  test[features]
y_test = test[target]

In [172]:
print(x_train.shape)
x_train.head()
# it dropped 8 columns. 

(10442, 13)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
66,1,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,1347,18,10022.0,0.0,0.0,0.0,0.0,1960.0,2
67,1,UPPER EAST SIDE (79-96),10 COOPS - ELEVATOR APARTMENTS,2,1491,62,10075.0,0.0,0.0,0.0,0.0,1925.0,2
71,1,UPPER WEST SIDE (59-79),13 CONDOS - ELEVATOR APARTMENTS,2,1171,2200,10069.0,1.0,0.0,1.0,827.0,2004.0,2


In [0]:
import category_encoders as ce

# instantiate category encoder to get one hot encoder. 
# link for future reference: https://contrib.scikit-learn.org/categorical-encoding/onehot.html
# category encoder link: https://contrib.scikit-learn.org/categorical-encoding/

encoder = ce.OneHotEncoder(use_cat_names = True)
x_train = encoder.fit_transform(x_train)
x_test = encoder.transform(x_test)

In [174]:
x_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_1,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_UPPER EAST SIDE (79-96),NEIGHBORHOOD_UPPER WEST SIDE (59-79),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_GRAMERCY,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_FOREST HILLS,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_10 COOPS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_13 CONDOS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_02 TWO FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_03 THREE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_41 TAX CLASS 4 - OTHER,BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_44 CONDO PARKING,BUILDING_CLASS_CATEGORY_07 RENTALS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_47 CONDO NON-BUSINESS STORAGE,BUILDING_CLASS_CATEGORY_05 TAX CLASS 1 VACANT LAND,BUILDING_CLASS_CATEGORY_43 CONDO OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_09 COOPS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_04 TAX CLASS 1 CONDOS,BUILDING_CLASS_CATEGORY_17 CONDO COOPS,BUILDING_CLASS_CATEGORY_29 COMMERCIAL GARAGES,BUILDING_CLASS_CATEGORY_15 CONDOS - 2-10 UNIT RESIDENTIAL,BUILDING_CLASS_CATEGORY_12 CONDOS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_27 FACTORIES,BUILDING_CLASS_CATEGORY_22 STORE BUILDINGS,BUILDING_CLASS_CATEGORY_31 COMMERCIAL VACANT LAND,BUILDING_CLASS_CATEGORY_45 CONDO HOTELS,BUILDING_CLASS_CATEGORY_32 HOSPITAL AND HEALTH FACILITIES,BUILDING_CLASS_CATEGORY_14 RENTALS - 4-10 UNIT,BUILDING_CLASS_CATEGORY_06 TAX CLASS 1 - OTHER,BUILDING_CLASS_CATEGORY_46 CONDO STORE BUILDINGS,BUILDING_CLASS_CATEGORY_21 OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_37 RELIGIOUS FACILITIES,BUILDING_CLASS_CATEGORY_33 EDUCATIONAL FACILITIES,BUILDING_CLASS_CATEGORY_30 WAREHOUSES,BUILDING_CLASS_CATEGORY_11A CONDO-RENTALS,BUILDING_CLASS_CATEGORY_36 OUTDOOR RECREATIONAL FACILITIES,BUILDING_CLASS_CATEGORY_34 THEATRES,BUILDING_CLASS_CATEGORY_16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT,BUILDING_CLASS_CATEGORY_26 OTHER HOTELS,BUILDING_CLASS_CATEGORY_28 COMMERCIAL CONDOS,BUILDING_CLASS_CATEGORY_35 INDOOR PUBLIC AND CULTURAL FACILITIES,BUILDING_CLASS_CATEGORY_39 TRANSPORTATION FACILITIES,BUILDING_CLASS_CATEGORY_42 CONDO CULTURAL/MEDICAL/EDUCATIONAL/ETC,BUILDING_CLASS_CATEGORY_23 LOFT BUILDINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_2,TAX_CLASS_AT_PRESENT_4,TAX_CLASS_AT_PRESENT_2A,TAX_CLASS_AT_PRESENT_1B,TAX_CLASS_AT_PRESENT_1A,TAX_CLASS_AT_PRESENT_2C,TAX_CLASS_AT_PRESENT_1C,TAX_CLASS_AT_PRESENT_2B,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
66,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1347,18,10022.0,0.0,0.0,0.0,0.0,1960.0,2
67,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1491,62,10075.0,0.0,0.0,0.0,0.0,1925.0,2
71,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1171,2200,10069.0,1.0,0.0,1.0,827.0,2004.0,2


In [175]:
x_train.shape

(10442, 75)

In [176]:
x_train.isna().sum().sort_values(ascending=False)[:10]
x_test.isna().sum().sort_values(ascending=False)[:10]
# year built has 5 nan values in x_train and 17 nan values in x_test
# i need to drop them in order to perform selectkbest

YEAR_BUILT                                                  17
BUILDING_CLASS_CATEGORY_43 CONDO OFFICE BUILDINGS            0
BUILDING_CLASS_CATEGORY_03 THREE FAMILY DWELLINGS            0
BUILDING_CLASS_CATEGORY_41 TAX CLASS 4 - OTHER               0
BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS     0
BUILDING_CLASS_CATEGORY_44 CONDO PARKING                     0
BUILDING_CLASS_CATEGORY_07 RENTALS - WALKUP APARTMENTS       0
BUILDING_CLASS_CATEGORY_47 CONDO NON-BUSINESS STORAGE        0
BUILDING_CLASS_CATEGORY_05 TAX CLASS 1 VACANT LAND           0
BUILDING_CLASS_CATEGORY_09 COOPS - WALKUP APARTMENTS         0
dtype: int64

In [0]:
# replace nan values with the mean of the column
x_train['YEAR_BUILT'] = x_train['YEAR_BUILT'].fillna(value =x_train['YEAR_BUILT'].mean() )
x_test['YEAR_BUILT'] = x_test['YEAR_BUILT'].fillna(value =x_test['YEAR_BUILT'].mean() )

In [0]:
# Do feature selection with SelectKBest

from sklearn.feature_selection import f_regression, SelectKBest

# There are 75 features, select 15 to begin with. 
selector = SelectKBest(score_func=f_regression, k=15)

# .fit_transform training data, then .transform testing data
x_train_selected = selector.fit_transform(x_train, y_train)
x_test_selected = selector.transform(x_test)

In [179]:
# which features where selected? This is for selecting individual features
names = x_train.columns
selected_features = selector.get_support()
selected_features_names = names[selected_features]
unselected_features_names = names[~selected_features]

print('Selected Features:')
for name in selected_features_names:
  print(name)

print('\nNot Selected Features:')
for name in unselected_features_names:
  print(name)

Selected Features:
BOROUGH_3
BOROUGH_4
BOROUGH_1
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
BUILDING_CLASS_CATEGORY_10 COOPS - ELEVATOR APARTMENTS
BUILDING_CLASS_CATEGORY_13 CONDOS - ELEVATOR APARTMENTS
BUILDING_CLASS_CATEGORY_03 THREE FAMILY DWELLINGS
BUILDING_CLASS_CATEGORY_07 RENTALS - WALKUP APARTMENTS
BUILDING_CLASS_CATEGORY_09 COOPS - WALKUP APARTMENTS
TAX_CLASS_AT_PRESENT_2
TAX_CLASS_AT_PRESENT_2A
BLOCK

Not Selected Features:
NEIGHBORHOOD_UPPER EAST SIDE (79-96)
NEIGHBORHOOD_UPPER WEST SIDE (59-79)
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_ASTORIA
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_GRAMERCY
NEIGHBORHOOD_UPPER EAST SIDE (59-79)
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_FOREST HILLS
BUILDING_CLASS_CATEGORY_02 TWO FAMILY DWELLINGS
BUILDING_CLASS_CATEGORY_41 TAX CLASS 4 - OTHER
BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS
BUILDING_CLASS_CATEGORY_44 CONDO PARKING
BUILDING_CLASS_CATEGORY_47 CONDO

In [180]:
# for selecting a model that has the "right" number of features
# the model(number of features) that presents itself with the lowest mean absolute error
# is the the best number of features(k) to use for this model. 

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

for k in range(1, len(x_train.columns)+1):
  print(f'{k} features')

  # do SelectKBest fit and transform
  selector = SelectKBest(score_func=f_regression, k=k)
  x_train_selected = selector.fit_transform(x_train, y_train)
  x_test_selected = selector.transform(x_test)

  # Do linear regression on the selected dataframes
  model = LinearRegression()
  model.fit(x_train_selected, y_train)

  # then predict the y which is price using the selected x_test features
  y_pred = model.predict(x_test_selected)
  error = mean_absolute_error(y_test, y_pred)
  print(f"Test Mean Absolute Error: ${error:,.0f}")

1 features
Test Mean Absolute Error: $312,606
2 features
Test Mean Absolute Error: $310,840
3 features
Test Mean Absolute Error: $282,638
4 features
Test Mean Absolute Error: $282,638
5 features
Test Mean Absolute Error: $276,889
6 features
Test Mean Absolute Error: $272,028
7 features
Test Mean Absolute Error: $269,495
8 features
Test Mean Absolute Error: $266,395
9 features
Test Mean Absolute Error: $265,021
10 features
Test Mean Absolute Error: $265,021
11 features
Test Mean Absolute Error: $251,900
12 features
Test Mean Absolute Error: $251,857
13 features
Test Mean Absolute Error: $251,370
14 features
Test Mean Absolute Error: $251,394
15 features
Test Mean Absolute Error: $246,984
16 features
Test Mean Absolute Error: $244,485
17 features
Test Mean Absolute Error: $244,428
18 features
Test Mean Absolute Error: $244,408
19 features
Test Mean Absolute Error: $243,460
20 features
Test Mean Absolute Error: $243,360
21 features
Test Mean Absolute Error: $243,336
22 features
Test Mean 

In [181]:
# feature scaling and ridge regression along with its error values. 

'''Ridge Regression is a technique for analyzing multiple regression data 
that suffer from multicollinearity. When multicollinearity occurs, 
least squares estimates are unbiased, but their variances are large 
so they may be far from the true value.'''

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from IPython.display import display, HTML
from ipywidgets import interact
import matplotlib.pyplot as plt

# we will start by trying different alpha values for ridge regression

for alpha in [10**1, 10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9]:

  # first standardize the data. we could also use 'ridge(normalize=True)' instead
  # of standardizing first. This is mostly if you want to standardize on your own
  # before fitting with ridge regression. 
  # standardization improves the condition of the problem and reduce variance. 

  scaler = StandardScaler()
  x_train_scaled = scaler.fit_transform(x_train)
  x_test_scaled = scaler.transform(x_test)

  # fitting the ridge regression model. 
  display(HTML(f'Ridge Regression, with alpha={alpha}'))
  model = Ridge(alpha=alpha)
  model.fit(x_train_scaled, y_train)

  # get the mean absolute error of the model for training data
  y_pred = model.predict(x_train_scaled)
  error = mean_absolute_error(y_train, y_pred)
  display(HTML(f'Train Mean Absolute Error: ${error:,.0f}'))

  # get the mean absolute error of the model for training data
  y_pred = model.predict(x_test_scaled)
  error = mean_absolute_error(y_test, y_pred)
  display(HTML(f'Test Mean Absolute Error: ${error:,.0f}'))

  # let't plot the coefficients from the ridge regression model. 
  # is not working for me and i don't know why. 
  coefficients = pd.Series(model.coef_, x_train.columns)
  plt.figure(figsize=(16,8))
  coefficients.sort_values().plot.barh(color='red')
  plt.xlim(-400, 1000)
  plt.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
